In [11]:
using Pkg
Pkg.activate(@__DIR__)
using SummationByParts, LinearAlgebra
using SummationByParts.Cubature, SummationByParts.SymCubatures

  Activating project at `~/Documents/codes/SummationByParts.jl/examples`


# Deriving Quadrature Rules on the Reference Tetrahedron

In the following examples, we demonstrate how one can use SummationByParts.jl to derive exiting or **new** quadrature rules on the reference tetrahedron. The input arguments required to construct an SBP operator are:

    - q            # degree of the quadrature 
    - vertices     # a boolean to indicate if the vertices symmetry group should be included
    - numS31       # an integer type for the number of S31 group
    - midedges     # a boolean to indicate if the midedge symmetry group should be included
    - numS22       # an integer type for the number of S22 group
    - numfaceS21   # an integer type for the number of S21 groups on the facet
    - numedge      # an integer type for the number of edge groups
    - numS211      # an integer type for the number of S211 group
    - numfaceS111  # an integer type for the number of S111 groups on the facet
    - facecentroid # a boolean to indicate if the centroid symmetry group on the facets should be included
    - numS1111     # an integer type for the number of the S1111 group
    - centroid     # a boolean to indicate if the centroid should be included
    - delta1       # perturbation coefficient when the objective function is <=0.1
    - delta2       # perturbation coefficient when the objective function is >0.1
    - verbose      # a boolean to show or hide interation results
    - xinit        # initial guess if available
    - xedge        # facet quadrature rule parameters (without weights) to construct SBP diagonal-E operators
    - xinit_sym_group # provides the ordering of the symmetry group in the xinit vector
    - xedge_sym_group # provides the ordering of the facet quadrature rule parameters in the xedge vector
Note: The following ordering is assumed in xinit and xedge vectors, unless otherwise provided: 

    [vertices,numS31,midedges,S22,faceS21,edge,S211,faceS111,facecentroid,S1111,centroid]

## Quadarature Rules for SBP-$\Omega$ Operators

Assume that we would like to derive a quadrature rule of degree 6 for an SBP-$\Omega$ operator. A known rule exists with numS31=3 and numS211=1. Hence, we set the solver as: 

In [12]:
cub, vtx = SummationByParts.deriveTetCubatureOmega(q=6,
                                                   numS31=3, 
                                                   numS22=0,
                                                   numS211=1, 
                                                   numS1111=0,
                                                   centroid=false,
                                                   xinit=[],
                                                   delta1=1e-3,
                                                   delta2=1e-1,
                                                   verbose=false)

No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
----------------------------------------------------------------------------------------------
iter_pso = 2400:  iter_lma = 414:  nperturb_pso = 20:  res norm = 1.9930359858831275e-15
----------------------------------------------------------------------------------------------


(TetSymCub{Float64}(5, 4, 24, false, false, false, false, 0, 0, 0, 3, 0, 1, 0, [0, 3, 0, 1, 0], [0.9670136704268264, 0.6438086137774565, 0.12202187560383404, 0.12732200375003513, 0.5393446629166315], [0.07380957539153972, 0.053230333677556564, 0.013436281407094184, 0.06428571428571432]), [-1.0 -1.0 -1.0; 1.0 -1.0 -1.0; -1.0 1.0 -1.0; -1.0 -1.0 1.0])

Initial guesses can be provided following the same approach as demonstrated for triangles. Furthermore, operations like accessing the weights or nodal locations in Cartesian coordinates are also carried out in a consistent manner.

## Quadarature Rules for SBP-$\Gamma$ Operators

Assume we want to construct a quadrature rule of degree 7, with vertices, midedge, and centroid. Further, assume it has numedge=1, numface21=1, numS31=1, and numS22=1. Then we set the solver as:

In [13]:
cub, vtx = SummationByParts.deriveTetCubatureGamma(q=7,
                                                   vertices=true,
                                                   numS31=1,
                                                   midedges=true, 
                                                   numS22=1,
                                                   numfaceS21=1, 
                                                   numedge=1, 
                                                   numS211=0,
                                                   numfaceS111=0, 
                                                   facecentroid=false,
                                                   numS1111=0,
                                                   centroid=true,
                                                   xinit=[],
                                                   delta1=1e-3,
                                                   delta2=1e-1,
                                                   verbose=false)

No solution found after a 200 LMA iterations.
----------------------------------------------------------------------------------------------
iter_pso = 800:  iter_lma = 212:  nperturb_pso = 1:  res norm = 2.5879278500950165e-15
----------------------------------------------------------------------------------------------


(TetSymCub{Float64}(4, 7, 45, true, true, true, false, 1, 1, 0, 1, 1, 0, 0, [1, 2, 2, 2, 0], [0.28418700275470193, 0.21742832019555544, 0.4500884831082469, 0.7426272531851915], [0.0015106273303336266, 0.06049054237435361, 0.00403888199622837, 0.10344930834722409, 0.024242961336136397, 0.005696088152131431, 0.08113091859465726]), [-1.0 -1.0 -1.0; 1.0 -1.0 -1.0; -1.0 1.0 -1.0; -1.0 -1.0 1.0])

## Quadrature Rules for SBP Diagonal-E operators

To derive a quadrature rule for an SBP diagonal-E operator, we must first find facet quadrature rules of degree at least $2p$ accurate, where $p$ is the degree of the SBP operator.

Assume we want to derive a degree $q=6$ quadrature rule (which enables constructing $p=3$ SBP diagonal-E operator). Then, we first find a degree $q_{f}=6$ facet quadrature rule. Assume this facet quadrature rule has the following symmetry groups: vertices=true, numedge=1, numS21=2. Then we obtain the facet quadratue parameters as:

In [14]:
qf = 6;
cub_facet, _ = SummationByParts.deriveTriCubatureGamma(q=qf,
                                                       vertices=true,
                                                       midedges=false,
                                                       numS21=2,
                                                       numedge=1,
                                                       numS111=0,
                                                       centroid=false,
                                                       xinit=[],
                                                       delta1=1e-3,delta2=1e-1,verbose=false)

No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
----------------------------------------------------------------------------------------------
iter_pso = 2400:  iter_lma = 411:  nperturb_pso = 30:  res norm = 2.008758959366972e-15
----------------------------------------------------------------------------------------------


(TriSymCub{Float64}(3, 4, 15, true, false, false, 1, 2, 0, [0, 3, 1], [0.23722737279318573, 0.8506802519794944, 0.3077459416259916], [0.014260718614408948, 0.20376930605390361, 0.33035897729113284, 0.05913883235361055]), [-1.0 -1.0; 1.0 -1.0; -1.0 1.0])

Then, we set the edge parameters for the quadrature on the tetrahedron as the parameters of the facet quadrature parameters computed above.

In [15]:
xedge = cub_facet.params
cub, vtx = SummationByParts.deriveTetCubatureDiagE(q=6,
                                                   vertices=true,
                                                   numS31=1,
                                                   midedges=false, 
                                                   numS22=1,
                                                   numfaceS21=2, 
                                                   numedge=1, 
                                                   numS211=0,
                                                   numfaceS111=0, 
                                                   facecentroid=false,
                                                   numS1111=0,
                                                   centroid=true,
                                                   xinit=[],
                                                   xedge=xedge,
                                                   delta1=1e-3,
                                                   delta2=1e-1,
                                                   verbose=false)

No solution found after a 200 LMA iterations.
----------------------------------------------------------------------------------------------
iter_pso = 800:  iter_lma = 212:  nperturb_pso = 0:  res norm = 1.9516100920050503e-15
----------------------------------------------------------------------------------------------


(TetSymCub{Float64}(5, 7, 51, true, false, true, false, 1, 2, 0, 1, 1, 0, 0, [1, 2, 1, 3, 0], [0.3931484668771776, 0.2578639185631969, 0.23722737279318573, 0.8506802519794944, 0.3077459416259916], [0.0008311992138364804, 0.06707451628396685, 0.0870062463078679, 0.014487003956737908, 0.025281032406421947, 0.004650827385021494, 0.0066466285167353575]), [-1.0 -1.0 -1.0; 1.0 -1.0 -1.0; -1.0 1.0 -1.0; -1.0 -1.0 1.0])

## Providing Initial Guess from the Literature

Note: When providing initial guesses from the literature, it might be necessary to scale the symmetry group parameters to correctly match the parameter ranges used in SummationByParts.jl. As an example, if one wants to provide the positive interior (PI) rules from Witherden and Vincent *(On the identification of symmetric quadrature rules for finite element methods, 2015)*, the following scalings are required:

- 3*numS31
- 2*numS22
- 2*numS211
- 2*numS1111 

Consider the degree 8 PI rule from Witherden and Vincent, which has numS31=4, numS22=1, and numS211=2. It can be provided as initial guess as follows:

In [17]:
xinit = [3*0.107952724962210848484,3*0.18510948778258657,3*0.04231654368476728,3*0.31418170912403899, #numS31
         2*0.43559132858383020, #numS22
         2*0.02143393012713057,2*0.23966807631943052,2*0.20413933387602912,2*0.00792395394579735, #numS211
         0.0352355345445451085, 0.0693756634183180436, 0.0100336748713869330, 0.0556850438092465302, #numS31 weights
         0.0483745736817450973, #numS22 weights
         0.0095425371877925775, 0.0206046482012804464]; #numS211 weights

In [18]:
cub, vtx = SummationByParts.deriveTetCubatureOmega(q=8,
                                                   numS31=4, 
                                                   numS22=1,
                                                   numS211=2, 
                                                   numS1111=0,
                                                   centroid=false,
                                                   xinit=xinit,
                                                   delta1=1e-3,
                                                   delta2=1e-1,
                                                   verbose=false)

----------------------------------------------------------------------------------------------
iter_pso = 0:  iter_lma = 0:  nperturb_pso = 0:  res norm = 1.954306892651273e-15
----------------------------------------------------------------------------------------------


(TetSymCub{Float64}(9, 7, 46, false, false, false, false, 0, 0, 0, 4, 1, 2, 0, [0, 4, 1, 2, 0], [0.32385817488663254, 0.5553284633477598, 0.12694963105430185, 0.9425451273721169, 0.8711826571676604, 0.04286786025426114, 0.47933615263886103, 0.40827866775205823, 0.0158479078915947], [0.03523553454454511, 0.06937566341831804, 0.010033674871386932, 0.05568504380924653, 0.048374573681745095, 0.009542537187792577, 0.020604648201280447]), [-1.0 -1.0 -1.0; 1.0 -1.0 -1.0; -1.0 1.0 -1.0; -1.0 -1.0 1.0])